In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW


In [2]:
# Load dataset
file_path = '/content/BenchmarkUddinSO-ConsoliatedAspectSentiment.xls'  # Update this to your file path
data = pd.read_excel(file_path)
df = data[['sent', 'ManualLabel', 'codes']]  # Adjust column names as per your dataset

# Filter for performance related entries
df_performance = df[df['codes'].str.contains('Legal', case=False)]

df_performance.loc[:, 'ManualLabel'] = df_performance['ManualLabel'].apply(lambda x: 0 if x == 'p' else (1 if x == 'n' else 2))

# Split the dataset into 60% training and 40% testing
train, test = train_test_split(df_performance, test_size=0.4, random_state=42)

print(f"Filtered data saved to new .xlsx file. Total entries with Community in code: {len(df_performance)}")

Filtered data saved to new .xlsx file. Total entries with Community in code: 50


In [3]:
# Tokenization
tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-tiny')
model = BertForSequenceClassification.from_pretrained('prajjwal1/bert-tiny')


class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings = tokenizer(train['sent'].tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test['sent'].tolist(), truncation=True, padding=True, max_length=512)

train_dataset = SentimentDataset(train_encodings, train['ManualLabel'].tolist())
test_dataset = SentimentDataset(test_encodings, test['ManualLabel'].tolist())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [5]:
!pip install accelerate -U

In [6]:
import torch
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, BertTokenizer
from torch.optim import AdamW

# Ensure the correct model and tokenizer are imported
tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-tiny')

# Model initialization with three classes: 'p', 'n', 'o' encoded as 0, 1, 2
model = BertForSequenceClassification.from_pretrained('prajjwal1/bert-tiny', num_labels=3)

# Data loaders for training and testing datasets
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Optimizer setup
optimizer = AdamW(model.parameters(), lr=5e-5)

# Setting device for model training (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop across specified number of epochs
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    # Iterating over batches
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Ensuring that labels are within the expected range [0, 2]
        assert labels.max() < 3 and labels.min() >= 0, "Labels are out of expected range."

        # Forward pass and loss calculation
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Calculating average loss for the epoch
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.5f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/15, Average Loss: 1.05307
Epoch 2/15, Average Loss: 1.03611
Epoch 3/15, Average Loss: 1.02157
Epoch 4/15, Average Loss: 1.00048
Epoch 5/15, Average Loss: 0.99592
Epoch 6/15, Average Loss: 0.94902
Epoch 7/15, Average Loss: 0.97593
Epoch 8/15, Average Loss: 0.96308
Epoch 9/15, Average Loss: 0.96333
Epoch 10/15, Average Loss: 0.93018
Epoch 11/15, Average Loss: 0.93077
Epoch 12/15, Average Loss: 0.91437
Epoch 13/15, Average Loss: 0.90724
Epoch 14/15, Average Loss: 0.88914
Epoch 15/15, Average Loss: 0.87900


In [7]:
import time
from sklearn.metrics import accuracy_score, f1_score

# Evaluation loop
model.eval()
predictions, true_labels = [], []
total_inference_time = 0
total_samples = 0

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        start_time = time.time()
        outputs = model(input_ids, attention_mask=attention_mask)
        inference_time = time.time() - start_time
        total_inference_time += inference_time
        total_samples += input_ids.size(0)

        logits = outputs.logits
        preds = logits.argmax(dim=1).cpu().numpy()
        labels = batch['labels'].cpu().numpy()

        predictions.extend(preds)
        true_labels.extend(labels)


# Compute metrics
accuracy = accuracy_score(true_labels, predictions)
f1_micro = f1_score(true_labels, predictions, average='micro')
f1_macro = f1_score(true_labels, predictions, average='macro')
f1_weighted = f1_score(true_labels, predictions, average='weighted')

print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score (Micro): {f1_micro:.4f}')
print(f'F1 Score (Macro): {f1_macro:.4f}')
print(f'F1 Score (Weighted): {f1_weighted:.4f}')

# Calculate average inference time per sample
average_inference_time = total_inference_time / total_samples
print(f'Total Inference Time: {total_inference_time:.6f} seconds')
print(f'Total Samples: {total_samples}')
print(f'Average Inference Time per Sample: {average_inference_time:.6f} seconds')


Accuracy: 0.4500
F1 Score (Micro): 0.4500
F1 Score (Macro): 0.2642
F1 Score (Weighted): 0.3270
Total Inference Time: 0.043555 seconds
Total Samples: 20
Average Inference Time per Sample: 0.002178 seconds
